In [1]:
#!npm install react react-dom @xyflow/react @anywidget/react

In [2]:
import sys
from pathlib import Path

sys.path.insert(0, str(Path(Path.cwd())))

In [3]:
!npx esbuild js/widget.jsx --minify --format=esm --bundle --outdir=pyironflow/static


  pyironflow/static/widget.js    1.7mb ⚠️
  pyironflow/static/widget.css  15.1kb

⚡ Done in 413ms


In [4]:
from pyiron_workflow import Workflow
from typing import Optional
from pyironflow.pyironflow import PyironFlow
from pyironflow.macro import get_node_instance_type

from pyiron_workflow import Workflow
from pyiron_workflow.channels import NotData
from pyironflow.themes import get_color
from pyironflow.macro import get_node_instance_type
from pyironflow.wf_extensions import get_import_path, create_macro

import anywidget
import pathlib
import traitlets
import os
import json
import importlib
import typing

In [5]:
wf = Workflow('lammps_macro')

In [6]:
from pyiron_workflow import as_function_node
import typing
from typing import Optional

@as_function_node()
def add_1(a: Optional[float | int] = None):
    return a+1

In [7]:
from pyiron_workflow import as_function_node, as_macro_node
import typing
from typing import Optional

@as_function_node()
def part_1(a: Optional[float | int] = 0):
    return a+1

@as_function_node()
def part_2(a: Optional[float | int] = 0):
    return a+2

@as_function_node()
def part_3(a: Optional[float | int] = 0):
    return a+3

@as_function_node()
def part_4(a: Optional[float | int] = 0, b: Optional[float | int] = 0):
    return a+b

@as_macro_node()
def macro_add(wf, a: int = 0):

    # imports

    wf.one = part_1(a)
    wf.two = part_2(wf.one)
    wf.three = part_3(wf.two)
    
    return wf.three

@as_macro_node()
def macro_addmore(wf, a: int = 0, b: int = 0):

    # imports

    wf.in1 = part_1(a)
    wf.in2 = part_2(b)
    wf.out = part_4(wf.in1, wf.in2)
    
    return wf.out
    


wf.add1 = add_1()
wf.add2 = add_1(wf.add1)
wf.add3 = add_1()
wf.add4 = add_1()

wf.macro1 = macro_addmore(wf.add2,wf.add3)
wf.macro2 = macro_add(wf.macro1)







In [8]:
pf = PyironFlow([wf])
pf.gui

In [11]:
list(wf.macro1.outputs.channel_dict.keys())[0]

'out'

In [15]:
for target in wf.macro1.outputs.channel_dict.keys():
    print(target)

out


In [ ]:
from pyiron_nodes.custom_macro import custom_macro

In [ ]:
    for i, (k, v) in enumerate(wf.children.items()):
        for ic, (out, inp) in enumerate(v.graph_as_dict['edges']['data']):
            print(ic, out, inp)
            last = max(len(out.split('/')) -1, 0) 
            out_node, out_port = out.split('/')[last].split('.')
            inp_node, inp_port = inp.split('/')[last].split('.')

In [ ]:
for i, (k, v) in enumerate(wf.children.items()):
    print(v.graph_as_dict)

In [ ]:
for i, (k, v) in enumerate(wf.children.items()):
    print(v)

In [ ]:
json.loads(pf.wf_widgets[0].gui.nodes)

In [9]:
json.loads(pf.wf_widgets[0].gui.nodes)

[{'id': 'add1',
  'data': {'label': 'add1',
   'source_labels': ['a+1'],
   'target_labels': ['a'],
   'import_path': '__main__.add_1',
   'target_values': [None],
   'target_types': ['float'],
   'source_values': ['NotData'],
   'source_types': ['None']},
  'position': {'x': 12, 'y': 136},
  'type': 'customNode',
  'style': {'border': '1px black solid',
   'padding': 5,
   'background': '#a2ea9f',
   'borderRadius': '10px',
   'width': '240px',
   'width_unitless': 240,
   'height_unitless': 56},
  'targetPosition': 'left',
  'sourcePosition': 'right',
  'measured': {'width': 240, 'height': 49}},
 {'id': 'add2',
  'data': {'label': 'add2',
   'source_labels': ['a+1'],
   'target_labels': ['a'],
   'import_path': '__main__.add_1',
   'target_values': [None],
   'target_types': ['float'],
   'source_values': ['NotData'],
   'source_types': ['None']},
  'position': {'x': 332, 'y': 88},
  'type': 'customNode',
  'style': {'border': '1px black solid',
   'padding': 5,
   'background': '#a2

In [ ]:
wf.plus1 = part_1()
wf.plus2 = part_2()
wf.plus3 = part_3()

In [ ]:
wf.add5 = add_1()

In [ ]:
wf.macro2 = macro_add(wf.macro1)

In [ ]:
wf.macro3 = custom_macro()

In [ ]:
pf.wf_widgets[0].update()

In [ ]:
json.loads(pf.wf_widgets[0].gui.nodes)

In [ ]:
@as_macro_node()
def macro_ception(wf, a: int = 0, b: int = 0):

    # imports

    #wf.one = part_1(a)
    #wf.two = part_2(b)
    wf.macro_inner = macro_addmore(a,b)
    wf.add_out = part_1(wf.macro_inner)
    
    return wf.add_out


In [ ]:
wf2 = Workflow('macro_in_macro')

wf2.macro1 = macro_ception()

pf2 = PyironFlow([wf2])
pf2.gui

In [ ]:
json.loads(pf.wf_widgets[0].gui.nodes)

In [ ]:
pf.wf_widgets[0].wf

In [ ]:
pf.wf_widgets[0].update()

In [ ]:
pf.wf_widgets[0].wf.add2 = add_1()

In [ ]:
wf.add1 = add_1()

In [ ]:
wf.add2 = add_1()

In [ ]:
pf.wf_widgets[0].add_node(add_1, "add1")

In [ ]:
pf.wf_widgets[0].gui.edges

In [ ]:
get_node_instance_type(wf.add1)

In [ ]:
get_edges(wf)

In [ ]:
for ic, (out, inp) in enumerate(wf.add1.graph_as_dict['edges']['data']):
    print(ic, out, inp)
    last = max(len(out.split('/')) -1, 0) 
    out_node, out_port = out.split('/')[last].split('.')
    print(out_node, out_port)

In [ ]:
for i, (k, v) in enumerate(wf.children.items()):
    print(k)

In [ ]:
def get_edges(wf):
    edges = []
    for ic, (out, inp) in enumerate(wf.graph_as_dict["edges"]["data"].keys()):
        out_node, out_port = out.split('/')[2].split('.')
        inp_node, inp_port = inp.split('/')[2].split('.')

        edge_dict = dict()
        edge_dict["source"] = out_node
        edge_dict["sourceHandle"] = out_port
        edge_dict["target"] = inp_node
        edge_dict["targetHandle"] = inp_port
        edge_dict["id"] = ic
        edge_dict["style"] = {'strokeWidth': 2, 'stroke': 'black',}

        edges.append(edge_dict)
    return edges


In [ ]:
get_edges(wf)

In [ ]:
edges = []
for i, (k, v) in enumerate(wf.children.items()):
    #print(v)
    #print(get_node_instance_type(v))
    if (get_node_instance_type(v) == 'macro'):
        for ic, (out, inp) in enumerate(v.graph_as_dict['edges']['data']):
            print(ic, out, inp)
            last = max(len(out.split('/')) -1, 0) 
            out_node, out_port = out.split('/')[last].split('.')
            inp_node, inp_port = inp.split('/')[last].split('.')
                
            edge_dict = dict()
            edge_dict["source"] = out_node
            edge_dict["sourceHandle"] = out_port
            edge_dict["target"] = inp_node
            edge_dict["targetHandle"] = inp_port
            edge_dict["id"] = ic
            edge_dict["style"] = {'strokeWidth': 2, 'stroke': 'black',}
    
            edges.append(edge_dict)
print(edges)

In [ ]:
for ic in enumerate(wf.graph_as_dict):
    print(ic)

In [ ]:
pf.wf_widgets[0].gui.edges

In [ ]:
pf.wf_widgets[0].gui.nodes = '[{"id":"add","data":{"label":"add"},"position":{"x":0,"y":100},"type":"group","style":{"border":"1px black solid","padding":5,"background":"#a2ea9f","borderRadius":"10px","width":"240px"},"measured":{"width":240,"height":12}},{"id":"add2","data":{"label":"add2"},"position":{"x":0,"y":100},"type":"custom_node", parentId: "add", extent: "parent", "style":{"border":"1px black solid","padding":5,"background":"#a2ea9f","borderRadius":"10px","width":"240px"},"measured":{"width":240,"height":12}}]'

In [ ]:
wf.macro = macro_add()

In [ ]:


flow_widget.add_node(str(path_str), node.path.name)


In [ ]:
from pyiron_workflow import as_function_node
import typing
from typing import Optional

@as_function_node()
def add_1(a: Optional[float | int] = None):
    return a+1

In [ ]:
wf.add1 = add_1()

In [ ]:
pf.wf_widgets[0].wf.add1 = add_1()

In [ ]:
wf.add2 = add_1()

In [ ]:
len(list(wf.macro))

In [ ]:
wf

In [ ]:
list(wf.macro)[0]

In [ ]:
import json

nodes = "[{'id': '1','data': { 'label': 'Hello' },'position': { 'x': 0, 'y': 0 },},]"

nodes_json = json.dumps(nodes)
wf.nodes = nodes_json

In [ ]:
wf

In [ ]:
    {
      'id': '1-a',
      'data': { 'label': 'message' },
      'position': { 'x': 250, 'y': 100 },
      'type': 'output',
      'parentId': '1',
      'extent': 'parent',
      'targetPosition': 'left',      
    },
    {
      'id': '1-b',
      'data': { 'label': 'Child Node 1' },
      'position': { 'x': 0, 'y': 100 },
      'type': 'input',
      'parentId': '1',
      'extent': 'parent',
      'targetPosition': 'left',  
    },

In [ ]:
Macro into default node

internal nodes

hidden button

connection validation

In [ ]:
no interactivity with expanded macro

In [ ]:
wf = pf.get_workflow()

In [ ]:
import anywidget
import pathlib
import traitlets
import os
import json
import importlib
import typing

def dict_to_node(dict_node):

    print(dict_node['type'])
    data = dict_node['data']
    node = get_node_from_path(data['import_path'])(label=dict_node['id'])
    if 'position' in dict_node:
        x, y = dict_node['position'].values()
        node.position = (x, y)
        # print('position exists: ', node.label, node.position)
    else:
        print('no position: ', node.label)
    if 'target_values' in data:
        target_values = data['target_values']
        target_labels = data['target_labels']
        for k, v in zip(target_labels, target_values):
            if v not in ('NonPrimitive', 'NotData'):
                node.inputs[k] = v

    return node

In [ ]:
dict_edges = json.loads(pf.wf_widgets[0].gui.edges)

In [ ]:
dict_edges

In [ ]:
for dict_edge in dict_edges:
    if 'type' in dict_edge:
        if (dict_edge['type'] != 'macroSubEdge'):
            print(dict_edge['type'])

In [ ]:
fruits = ['apple', 'banana', 'cherry']

fruits.count("chry")